# MetaNetX data preprocessing

Correlate `reac_prop.tsv` and `chem_prop.tsv` to convert dataset into useful format (SMILES, InChI or InChiKey).

Two methods, one simpler and one more comprehensive.

1. Discard all reaction information and get a list of metabolites involved in reaction to be seen as "bioreachable".
2. Retain all reaction information but change MNX_ID's into useful chemical representations.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [2]:
reac = pd.read_csv('reac_prop.tsv', sep='\t', header=351) # skip 351 lines of documentation
reac.head()

,#ID,mnx_equation,reference,classifs,is_balanced,is_transport
0,EMPTY,=,mnx:EMPTY,NaN,B,NaN
1,MNXR01,1 MNXM01@MNXD1 = 1 MNXM1@MNXD1,mnx:MNXR01,NaN,B,NaN
2,MNXR02,1 MNXM1@MNXD1 = 1 MNXM1@MNXD2,mnx:MNXR02,NaN,B,T
3,MNXR03,1 MNXM01@MNXD1 = 1 MNXM01@MNXD2,mnx:MNXR03,NaN,B,T
4,MNXR100000,1 MNXM10958@MNXD1 + 1 MNXM1104529@MNXD1 = 1 MN...,biggR:GALNACT5g,NaN,NaN,NaN


## Clean `reac_prop.tsv`

1. Discard compartment information such as "@MNXD1".
2. Discard useless columns "reference", "classifs", "is_balanced" and "is_transport".
3. Re-organize reactions into 4 formats:
    1. A + B = C + D (As it is)
    2. list of metabolites involved in the reaction (removing duplicates)
    3. list of substrates
    4. list of products

In [3]:
def preprocess_rex(rex):
    """
    rex: one reaction, such as '1 MNXM01@MNXD1 = 1 MNXM1@MNXD1'

    output: 
        - reaction formula without compartment (MNXM01 instead of MNXM01@MNXD1)
        - list of metabolites involved in the reaction
        - list of substrates
        - list of products
    """
    # compartment can only be @MNXD1 or @MNXD2
    rex_clean = rex.replace('@MNXD1', '').replace('@MNXD2', '')

    metabolites = take_MNXM(rex_clean.split(' '))
    # drop duplicates
    metabolites = list(set(metabolites))

    substrates, products = rex_clean.split('=')
    substrates = take_MNXM(substrates.split(' '))
    products = take_MNXM(products.split(' '))

    return rex_clean, metabolites, substrates, products


def take_MNXM(str_list):
    """
    Helper function for preprocessing.
    """
    return [mol for mol in str_list if mol.startswith('MNXM')]

In [4]:
reac_prep = []
for rex in reac['mnx_equation']:
    reac_prep.append(preprocess_rex(rex))
reac_prep = np.array(reac_prep, dtype=object)

reac = reac[['#ID', 'mnx_equation']]
reac['equation'] = reac_prep[:, 0]
reac['metabolites'] = reac_prep[:, 1]
reac['substrates'] = reac_prep[:, 2]
reac['products'] = reac_prep[:, 3]

reac.head()

,#ID,mnx_equation,equation,metabolites,substrates,products
0,EMPTY,=,=,[],[],[]
1,MNXR01,1 MNXM01@MNXD1 = 1 MNXM1@MNXD1,1 MNXM01 = 1 MNXM1,"[MNXM01, MNXM1]",[MNXM01],[MNXM1]
2,MNXR02,1 MNXM1@MNXD1 = 1 MNXM1@MNXD2,1 MNXM1 = 1 MNXM1,[MNXM1],[MNXM1],[MNXM1]
3,MNXR03,1 MNXM01@MNXD1 = 1 MNXM01@MNXD2,1 MNXM01 = 1 MNXM01,[MNXM01],[MNXM01],[MNXM01]
4,MNXR100000,1 MNXM10958@MNXD1 + 1 MNXM1104529@MNXD1 = 1 MN...,1 MNXM10958 + 1 MNXM1104529 = 1 MNXM1102128 + ...,"[MNXM1102128, MNXM8415, MNXM1104529, MNXM10958]","[MNXM10958, MNXM1104529]","[MNXM1102128, MNXM8415]"


In [5]:
chem = pd.read_csv('chem_prop.tsv', sep='\t', header=351) # skip 351 lines of documentation
chem.head()

,#ID,name,reference,formula,charge,mass,InChI,InChIKey,SMILES
0,BIOMASS,BIOMASS,mnx:BIOMASS,NaN,NaN,NaN,NaN,NaN,NaN
1,MNXM01,PMF,mnx:PMF,H,1.0,1.00794,InChI=1S/p+1,InChIKey=GPRLSGONYQIRFK-UHFFFAOYSA-N,[H+]
2,MNXM02,OH(-),mnx:HYDROXYDE,OH,-1.0,17.00734,InChI=1S/H2O/h1H2/p-1,InChIKey=XLYOFNOQVPJJNP-UHFFFAOYSA-M,[O-][H]
3,MNXM03,H3O(+),mnx:OXONIUM,H3O,1.0,19.02322,InChI=1S/H2O/h1H2/p+1,InChIKey=XLYOFNOQVPJJNP-UHFFFAOYSA-O,[OH3+]
4,MNXM1,H(+),mnx:PROTON,H,1.0,1.00794,InChI=1S/p+1,InChIKey=GPRLSGONYQIRFK-UHFFFAOYSA-N,[H+]


## ALL metabolites without RXN info

In [26]:
metabolites_list = []
for i in trange(len(reac)):
    metabolites_list += reac.loc[i, 'metabolites']
metabolites_list = list(set(metabolites_list))
bioreachable = chem[chem['#ID'].isin(metabolites_list)]
bioreachable.index = range(len(bioreachable))
bioreachable

100%|██████████| 74613/74613 [00:01<00:00, 62496.60it/s]


,#ID,name,reference,formula,charge,mass,InChI,InChIKey,SMILES
0,MNXM01,PMF,mnx:PMF,H,1.0,1.00794,InChI=1S/p+1,InChIKey=GPRLSGONYQIRFK-UHFFFAOYSA-N,[H+]
1,MNXM1,H(+),mnx:PROTON,H,1.0,1.00794,InChI=1S/p+1,InChIKey=GPRLSGONYQIRFK-UHFFFAOYSA-N,[H+]
2,MNXM10,NADH,chebi:57945,C21H27N7O14P2,-2.0,663.11022,InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17...,InChIKey=BOPGDPNILDQYTO-NNYOXOHSSA-L,NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O)(...
3,MNXM100,(2E)-geranyl diphosphate,chebi:58057,C10H17O7P2,-3.0,311.04660,InChI=1S/C10H20O7P2/c1-9(2)5-4-6-10(3)7-8-16-1...,InChIKey=GVVPGTZRZFNKDS-JXMROGBWSA-K,CC(C)=CCC/C(C)=C/COP(=O)([O-])OP(=O)([O-])[O-]
4,MNXM10002,3-deoxycapsidiol,chebi:72642,C15H24O,0.0,220.18272,InChI=1S/C15H24O/c1-10(2)12-6-7-13-14(16)8-5-1...,InChIKey=NJWPLFBOSCSZFA-QHSBEEBCSA-N,C=C(C)[C@@H]1CC=C2[C@H](O)CC[C@@H](C)[C@@]2(C)C1
...,...,...,...,...,...,...,...,...,...
42547,MNXM9994,"5-chlorobenzoate-cis-3,4-diol",metacycM:CPD-11220,C7H6ClO4,-1.0,188.99601,InChI=1S/C7H7ClO4/c8-4-1-3(7(11)12)2-5(9)6(4)1...,InChIKey=GNYUNLRRAAQENB-NTSWFWBYSA-M,O=C([O-])C1=C[C@H](O)[C@H](O)C(Cl)=C1
42548,MNXM9995,3-chlorotoluene,metacycM:CPD-10654,C7H7Cl,0.0,126.02363,"InChI=1S/C7H7Cl/c1-6-3-2-4-7(8)5-6/h2-5H,1H3",InChIKey=OSOUNOBYRMOXQQ-UHFFFAOYSA-N,Cc1cccc(Cl)c1
42549,MNXM99969,compound 0043171,envipathM:650babc9-9d68-4b73-9332-11972ca26f7b...,C35H72,0.0,492.56340,InChI=1S/C35H72/c1-3-5-7-9-11-13-15-17-19-21-2...,InChIKey=VHQQPFLOGSTQPC-UHFFFAOYSA-N,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
42550,MNXM9999,3-demethylubiquinol-7,chebi:84431,C43H66O4,0.0,646.49611,InChI=1S/C43H66O4/c1-31(2)17-11-18-32(3)19-12-...,InChIKey=OHBHBMXNJCUMCR-DKCCAHEHSA-N,COc1c(O)c(O)c(C)c(C/C=C(\C)CC/C=C(\C)CC/C=C(\C...


In [27]:
bioreachable.to_csv('mnx_chem_bioreachable.csv', index=False)

## Retaining RXN info

In [12]:
def findIdentifier(mnxid_list, identifier='SMILES'):

    identifiers_list = []
    for mnxid in tqdm(mnxid_list):
        identifiers_list.append(chem[chem['#ID'] == mnxid][identifier])
    
    return identifiers_list

In [ ]:
for i in trange(len(reac)):

    substrates_identifiers = findIdentifier(reac.loc[i, ])